<H1>Test Notebook for Stephenson System</H1>

In [30]:
import pyreadr
import pandas as pd
import numpy as np
import datetime
import time
import copy
from stephenson import *

Read R file and convert to pandas df

In [31]:
result = pyreadr.read_r('../data/aflodds.rda')
r_df = list(result.values())[0]
df = pd.DataFrame(r_df)
df.head()

,Date,Week,HomeTeam,AwayTeam,HomeScore,AwayScore,Score,HomeOdds,AwayOdds
0,2009-03-26,1.0,Richmond Tigers,Carlton Blues,67,150,0.0,NaN,NaN
1,2009-03-27,1.0,Hawthorn Hawks,Geelong Cats,103,111,0.0,NaN,NaN
2,2009-03-28,1.0,Collingwood Magpies,Adelaide Crows,86,90,0.0,NaN,NaN
3,2009-03-28,1.0,Brisbane Lions,West Coast Eagles,95,86,1.0,NaN,NaN
4,2009-03-28,1.0,St Kilda Saints,Sydney Swans,80,65,1.0,NaN,NaN


From there select week one outcomes, and create a dataframe of teams and their respective stephenson ratings

In [32]:
weeks = df[(df['Week'] <= 12.0)]

combined_teams = pd.concat([weeks['HomeTeam'], weeks['AwayTeam']]) 
all_teams = combined_teams.drop_duplicates().tolist()

team_data = {
    'team': all_teams,   
    'steph_rating': [Stephenson() for _ in all_teams],
}
df_team = pd.DataFrame(team_data)
df_team.head()

,team,steph_rating
0,Richmond Tigers,<stephenson.Stephenson object at 0x12dfefb00>
1,Hawthorn Hawks,<stephenson.Stephenson object at 0x12dfecf20>
2,Collingwood Magpies,<stephenson.Stephenson object at 0x12dfed040>
3,Brisbane Lions,<stephenson.Stephenson object at 0x12dfef3e0>
4,St Kilda Saints,<stephenson.Stephenson object at 0x12dff5910>


Since this is the first outcomes for each team, we don't have a need to update the variance prior to the matchup, however, if this was any other time, uncomment the updateVar with relevant dates. -- Make sure to adjust the rating period to one that suits your system. From there we will adjust the ratings based on the outcome 1 for win 0 for loos .5 for tie. We clone the home rating so when we calculate the away rating we don't use the updated home rating. Then we assign these new ratings to the df

In [ ]:
for week, week_df in weeks.groupby('Week'):
    # print(f"Week {week}")
    for index, row in week_df.iterrows():
        home_rating = df_team.loc[df_team['team'] == row['HomeTeam'], 'steph_rating'].values[0]
        away_rating = df_team.loc[df_team['team'] == row['AwayTeam'], 'steph_rating'].values[0]

        #Replacing updateVal since this is not by date but by week 10^2 = 100 (cval)
        home_rating.sigma = home_rating.sigma + 100
        away_rating.sigma = away_rating.sigma + 100

        # So we don't evaluate on outcome
        home_clone = copy.deepcopy(home_rating)

        home_rating.newVarRating(away_rating, row['Score'], 1) #pone is 1 for home team - since default gamma is 0 this is irrelevant
        away_rating.newVarRating(home_clone, (1 - row['Score']), 0) 

        df_team.loc[df_team['team'] == row['HomeTeam'], 'steph_rating'] = home_rating
        df_team.loc[df_team['team'] == row['AwayTeam'], 'steph_rating'] = away_rating

Week 1.0
Week 2.0
Week 3.0
Week 4.0
Week 5.0
Week 6.0
Week 7.0
Week 8.0
Week 9.0
Week 10.0
Week 11.0
Week 12.0


Finally just a test print to show that it returns same results as in R.

In [34]:
for index, row in df_team.iterrows():
    print(f"{row['team']}:", end=" ")
    row['steph_rating'].printVals()

Richmond Tigers: Rating: 2034.775979625707 Deviation: 126.77282448987641
Hawthorn Hawks: Rating: 2202.662500420687 Deviation: 117.0812173422675
Collingwood Magpies: Rating: 2225.6291469613225 Deviation: 123.25442062398865
Brisbane Lions: Rating: 2291.3948426943807 Deviation: 119.26363716004963
St Kilda Saints: Rating: 2592.0674606569096 Deviation: 142.74260172410345
Melbourne Demons: Rating: 1909.5491712911846 Deviation: 139.2051565455552
Port Adelaide Power: Rating: 2132.837013035284 Deviation: 118.50036544417276
Fremantle Dockers: Rating: 2012.1296362236876 Deviation: 124.94252513010751
Adelaide Crows: Rating: 2277.020478115515 Deviation: 115.58151788535875
Geelong Cats: Rating: 2555.2882249795753 Deviation: 145.6526818926217
Carlton Blues: Rating: 2215.1123766821775 Deviation: 118.57448824802964
Sydney Swans: Rating: 2158.992904743084 Deviation: 121.60371574752281
Essendon Bombers: Rating: 2198.244240667883 Deviation: 126.86098827732206
Western Bulldogs: Rating: 2325.5452602666605 D